In [1]:
# Cell 1: Importing Required Libraries

import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Adding src directory to path for importing custom modules
sys.path.append('../src')

# Importing custom utility functions
from config import *
from data_utils import *
from visualization import *

# Setting visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")

# Displaying settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

print("All libraries imported successfully!")
print(f"Working Directory: {Path.cwd()}")
print(f"Random Seed: {SEED_VALUE}")

All libraries imported successfully!
Working Directory: C:\Users\Ashutosh\Documents\Projects\beyond-smote-evaluation\notebooks
Random Seed: 42


In [4]:
# Cell 2: Dataset Overview and Citation

print("HIGGS Dataset Information")
print("\nDataset Citation")
print("Whiteson, D. (2014). HIGGS [Dataset]. UCI Machine Learning Repository.")
print("https://doi.org/10.24432/C5V312")
print("\nOriginal Paper:")
print("Baldi, P., Sadowski, P., & Whiteson, D. (2014).")
print("Searching for exotic particles in high-energy physics with deep learning.")
print("Nature Communications, 5(1), 4308.")

print("\nDataset Specifications")
print(f"  • Total Rows: {TOTAL_ROWS:,}")
print(f"  • Total Features: {FEATURE_COUNT}")
print(f"  • Target Classes: {CLASS_COUNT} (Binary Classification)")
print(f"  • File Size: ~8 GB uncompressed")
print(f"  • Source: UCI Machine Learning Repository")
print(f"  • Domain: Particle Physics (CERN)")
print(f"  • Task: Distinguish Higgs boson signal from background")

print("\nProblem Statement:")
print("To classify particle collision events as signal (presence of Higgs boson)")
print("or background (absence of Higgs boson) using 28 physics-based features.")

HIGGS Dataset Information

Dataset Citation
Whiteson, D. (2014). HIGGS [Dataset]. UCI Machine Learning Repository.
https://doi.org/10.24432/C5V312

Original Paper:
Baldi, P., Sadowski, P., & Whiteson, D. (2014).
Searching for exotic particles in high-energy physics with deep learning.
Nature Communications, 5(1), 4308.

Dataset Specifications
  • Total Rows: 11,000,000
  • Total Features: 28
  • Target Classes: 2 (Binary Classification)
  • File Size: ~8 GB uncompressed
  • Source: UCI Machine Learning Repository
  • Domain: Particle Physics (CERN)
  • Task: Distinguish Higgs boson signal from background

Problem Statement:
To classify particle collision events as signal (presence of Higgs boson)
or background (absence of Higgs boson) using 28 physics-based features.
